[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Scientific Programming Methods

## SVD & Linear Least Squares - SVD Rank Approximation

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 12/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0012LinearFitL1.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import njit

# Machine Learning

# Optimization

# Image Processing / Computer Vision
import skimage as ski

# Miscellaneous
import math
from platform import python_version
import random
import time

# Typing
from typing import Callable, List, Optional, Tuple, Union

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions

# @njit(cache = True)
def RowToImg( mR: np.ndarray, mI: np.ndarray, tuBlockSize: Tuple[int, int] ):
    """
    Rearranges a 2D array of flattened blocks (rows) into a 2D image array by placing
    each block into its corresponding position within the image. Matches MATLAB's `col2im()` 
    function in `'distinct'` mode.

    Parameters
    ----------
    mR : np.ndarray
        A 2D NumPy array where each row represents a flattened block of size `tuBlockSize`.
        The number of rows in `mR` corresponds to the total number of blocks in the image.
    mI : np.ndarray
        A 2D NumPy array that serves as the output image array to which blocks will be 
        placed. The shape of `mI` must match the dimensions implied by the block size and 
        the number of rows in `mR`.
    tuBlockSize : Tuple[int, int]
        A tuple `(blockHeight, blockWidth)` specifying the dimensions of each block.

    Returns
    -------
    mI : np.ndarray
        A 2D NumPy array that serves as the output image array to which blocks will be 
        placed. The shape of `mI` must match the dimensions implied by the block size and 
        the number of rows in `mR`.

    Raises
    ------
    ValueError
        If the dimensions of `mR` and `mI` are incompatible with the specified block size.

    Notes
    -----
    - The function assumes non-overlapping blocks, and the shape of `mI` must be divisible 
      by the block size `(blockHeight, blockWidth)`. No compatibility checks are performed 
      within the function.
    - This function is the inverse of block extraction functions like MATLAB's `im2col()` 
      with the `'distinct'` mode or Python equivalents like `ImgBlockView()`.

    Examples
    --------
    Consider reconstructing an 8x8 image from flattened 4x4 blocks:

    >>> import numpy as np
    >>> mR = np.array([[ 0,  1,  2,  3,  8,  9, 10, 11, 16, 17, 18, 19, 24, 25, 26, 27],
    ...                [ 4,  5,  6,  7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31],
    ...                [32, 33, 34, 35, 40, 41, 42, 43, 48, 49, 50, 51, 56, 57, 58, 59],
    ...                [36, 37, 38, 39, 44, 45, 46, 47, 52, 53, 54, 55, 60, 61, 62, 63]])
    >>> mI = np.zeros((8, 8), dtype=int)
    >>> tuBlockSize = (4, 4)
    >>> RowToImg(mR, mI, tuBlockSize)
    >>> print(mI)
    [[ 0  1  2  3  4  5  6  7]
     [ 8  9 10 11 12 13 14 15]
     [16 17 18 19 20 21 22 23]
     [24 25 26 27 28 29 30 31]
     [32 33 34 35 36 37 38 39]
     [40 41 42 43 44 45 46 47]
     [48 49 50 51 52 53 54 55]
     [56 57 58 59 60 61 62 63]]
    """
    
    # https://github.com/numba/numba/issues/9464
    numRows = np.size(mI, 0)
    numCols = np.size(mI, 1)
    
    kk = 0
    for ii in range(0, numRows, tuBlockSize[0]):
        for jj in range(0, numCols, tuBlockSize[1]):
            mI[ii:(ii + tuBlockSize[0]), jj:(jj + tuBlockSize[1])].flat = mR[kk] #<! https://github.com/numba/numba/issues/10070
            kk += 1
    
    return mI


In [ ]:
# Parameters

# Data
# imgUrl = 'https://i.imgur.com/3BbIXdH.png' #<! A turtle climbing the Everest!\n",
imgUrl = 'https://i.postimg.cc/63rN33GZ/3BbIXdH.png' #<! A turtle climbing the Everest!\n",
paramK = 8 #<! Working on patches with size (paramK, paramK)

# Model
lSR    = [0, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 60, 64] #<! Number of singular values for reconstruction
numRec = len(lSR)


## SVD for Image Compression

The SVD can be used for [Low Rank Approximation](https://en.wikipedia.org/wiki/Low-rank_approximation) of a matrix.  
In the context of image processing, the image patches are composing a matrix $\boldsymbol{D}$.  
By the [SVD Theorem](https://en.wikipedia.org/wiki/Singular_value_decomposition) $\boldsymbol{D} = \boldsymbol{U} \boldsymbol{S} \boldsymbol{V}^{T}$ which implies $\boldsymbol{U}^{T} \boldsymbol{D} = \boldsymbol{S} \boldsymbol{V}^{T}$ or $\boldsymbol{D} \boldsymbol{V} = \boldsymbol{U} \boldsymbol{S}$.

The concept is to code the image using only part of the singular values which is a low rank approximation of the data.

* <font color='brown'>(**#**)</font> The process above is approximation of the Karhunen Loeve Transform (See [Karhunen Loeve Theorem](https://en.wikipedia.org/wiki/Kosambi%E2%80%93Karhunen%E2%80%93Lo%C3%A8ve_theorem)).
* <font color='brown'>(**#**)</font> The KLT is closely related to the [Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA).  
  See [What Is the Difference Between PCA and Karhunen Loeve (KL) Transform](https://dsp.stackexchange.com/questions/49210).
* <font color='brown'>(**#**)</font> For natural patches, it can be shown that the KLT can be well approximated by the [Discrete Cosine Transform](https://en.wikipedia.org/wiki/Discrete_cosine_transform) (DCT).  
  See [Discrete Cosine Transform (DCT) as the Limit of Principal Component Analysis (PCA)](https://dsp.stackexchange.com/questions/86375).
* <font color='brown'>(**#**)</font> The DCT is used in many image compression algorithms. See [JPEG](https://en.wikipedia.org/wiki/JPEG).


## Generate Data


Load the image.

In [ ]:
# Generate / Load the Data

# Image
mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float64(mI)
mI = np.mean(mI, axis = 2)
mI = ski.transform.resize(mI, output_shape = (paramK * 80, paramK * 80), preserve_range = False)


numRows = np.size(mI, 0)
numCols = np.size(mI, 1)

tR = np.zeros(shape = (len(lSR), numRows, numCols)) #<! Reconstructed images


In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (10, 6))
hA.imshow(mI, cmap = 'gray', vmin = 0.0, vmax = 1.0)
hA.set_title(f'Image of Size: ({mI.shape[0]}, {mI.shape[1]})');

* <font color='red'>(**?**)</font> Given the parameter `paramK` and image of size as above. What would be the size of the array where each patch is a column?

## The `im2col()` Operator

The operator, coined mainly by MATLAB's [`im2col()`](https://www.mathworks.com/help/images/ref/im2col.html), builds an array where each column is a patch.  
NumPy allows building such array using views which on one hand more efficient in memory yet on the other hand the data locality is less optimized. 

In [ ]:
# Objective Function

#===========================Fill This===========================#
# 1. Implement the `ImgBlockView()` function. 
#    Given a vector of `vX` it returns the objective.
# 2. The implementation should be using a Lambda Function.
# !! You may `np.square()` and / or `np.linalg.norm()`.

def ImgBlockView(mI: np.ndarray, tuBlockShape: Tuple[int, int] = (4, 4)) -> np.ndarray:
    """
    Divides a 2D input array into non overlapping blocks of a specified shape 
    and returns a flattened view of these blocks as rows of a 2D array.

    Parameters
    ----------
    mI : np.ndarray
        A 2D NumPy array (Image or Matrix) to be divided into blocks.
    tuBlockShape : Tuple[int, int], optional
        The shape of the blocks (rows, columns) to divide the input array into.
        Defaults to `(4, 4)`.

    Returns
    -------
    np.ndarray
        A 2D NumPy array where each row represents a flattened view of a block 
        from the input array. Might be a copy of the data and not a view.

    Raises
    ------
    ValueError
        If the shape of `mI` is not an integer multiple of the block shape `tuBlockShape`.

    Notes
    -----
    - The function uses NumPy's `stride_tricks()` to efficiently generate views of the 
      input array without creating copies of the data.
    - The shape of the input array `mI` must be an exact multiple of the block shape 
      specified by `tuBlockShape`. No shape compatibility checks are performed.
    - The output is similar to MATLAB's `im2col()` function with the `'distinct'` mode, 
      where non overlapping blocks are extracted.

    Examples
    --------
    Divide a 6x6 matrix into 3x3 blocks:
    
    >>> import numpy as np
    >>> mI = np.arange(36).reshape(6, 6)
    >>> print(mI)
    [[ 0  1  2  3  4  5]
     [ 6  7  8  9 10 11]
     [12 13 14 15 16 17]
     [18 19 20 21 22 23]
     [24 25 26 27 28 29]
     [30 31 32 33 34 35]]
    >>> tuBlockShape = (3, 3)
    >>> mB = ImgBlockView(mI, tuBlockShape)
    >>> print(mB)
    [[ 0  1  2  6  7  8 12 13 14]
     [ 3  4  5  9 10 11 15 16 17]
     [18 19 20 24 25 26 30 31 32]
     [21 22 23 27 28 29 33 34 35]]

    Divide an 8x8 array into 4x4 blocks:
    
    >>> mI = np.arange(64).reshape(8, 8)
    >>> print(mI)
    [[ 0  1  2  3  4  5  6  7]
     [ 8  9 10 11 12 13 14 15]
     [16 17 18 19 20 21 22 23]
     [24 25 26 27 28 29 30 31]
     [32 33 34 35 36 37 38 39]
     [40 41 42 43 44 45 46 47]
     [48 49 50 51 52 53 54 55]
     [56 57 58 59 60 61 62 63]]
    >>> tuBlockShape = (4, 4)
    >>> mB = ImgBlockView(mI, tuBlockShape)
    >>> print(mB)
    [[ 0  1  2  3  8  9 10 11 16 17 18 19 24 25 26 27]
     [ 4  5  6  7 12 13 14 15 20 21 22 23 28 29 30 31]
     [32 33 34 35 40 41 42 43 48 49 50 51 56 57 58 59]
     [36 37 38 39 44 45 46 47 52 53 54 55 60 61 62 63]]
    """
    # Pay attention to integer division
    # Tuple addition means concatenation of the Tuples
    tuShape   = (mI.shape[0] // tuBlockShape[0], mI.shape[1] // tuBlockShape[1]) + tuBlockShape
    tuStrides = (tuBlockShape[0] * mI.strides[0], tuBlockShape[1] * mI.strides[1]) + mI.strides

    tA = np.lib.stride_tricks.as_strided(mI, shape = tuShape, strides = tuStrides) #<! (numPatchW, numPatchH, tuBlockShape[0], tuBlockShape[1])

    mB = np.reshape(tA, ((mI.shape[0] // tuBlockShape[0]) * (mI.shape[1] // tuBlockShape[1]), tuBlockShape[0] * tuBlockShape[1])) #<! Creates a copy!
    
    return mB
#===============================================================#

* <font color='brown'>(**#**)</font> In MATLAB's `im2co()` each patch is contiguous in memory. The view above are not.

In [ ]:
# The Patch Block View

mD = ImgBlockView(mI, (paramK, paramK)) #<! Each row is a block / patch


In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (10, 6))
hA.imshow(mD, cmap = 'gray', vmin = 0.0, vmax = 1.0)
hA.set_title(f'Block Array of Size: ({mD.shape[0]}, {mD.shape[1]})');

## SVD of Data




### The PCA / SVD Pre Processing

1. Remove the mean patch from all patches.
2. Apply the SVD to the data matrix.

In [ ]:
# SVD and Singular Value Distribution

#===========================Fill This===========================#
# 1. Calculate the mean patch from the data.
# 2. Make the data with zero mean.
# 3. Calculate the SVD of the centered data.

vMeanD      = np.mean(mD, axis = 0)
mD          = mD - vMeanD
mU, vS, mVh = np.linalg.svd(mD)
#===============================================================#


### The Distribution of the Singular Values

In [ ]:
# The Distribution of the Singular Values

numSingVal  = len(vS)
vSEleEnergy = vS / np.sum(vS)
vSAccEnergy = np.cumsum(vS) / sum(vS)


In [ ]:
# Display Data

hF, hA = plt.subplots(figsize = (10, 6))
hA.scatter(range(numSingVal), vSEleEnergy, s = 35, label = 'Normalized Energy')
hA.scatter(range(numSingVal), vSAccEnergy, s = 10, label = 'Accumulated Energy')
hA.set_title(f'Singular Values Distribution, #{numSingVal}')
hA.set_xlabel('Index')
hA.set_ylabel('Normalized Value')

hA.legend();

### Projection onto the Columns / Rows Space

In [ ]:
# Projection onto the Columns Space

mRecPatches = np.zeros_like(mD) #<! Reconstructed patches

for ii in range(numRec):
    recRank = lSR[ii]
    mUd = mU[:, :recRank]
    mRecPatches[:] = mUd @ (mUd.T @ mD) + vMeanD
    tR[ii] = RowToImg(mRecPatches, tR[ii], (paramK, paramK))


In [ ]:
# Projection onto the Row Space

mRecPatches = np.zeros_like(mD) #<! Reconstructed patches

for ii in range(numRec):
    recRank = lSR[ii]
    mVd = mVh.T[:, :recRank]
    # mRecPatches[:] = mUd @ (mUd.T @ mD) + vMeanD
    mRecPatches[:] = (mD @ mVd) @ mVd.T + vMeanD
    tR[ii] = RowToImg(mRecPatches, tR[ii], (paramK, paramK))

In [ ]:
# Display Image Array

def DisplayImageArray( tI: np.ndarray, imgIdx: int, /, *, figTitle: Optional[str] = None ) -> plt.Figure:

    hF, hA = plt.subplots(figsize = (8, 6))

    hA.imshow(tI[imgIdx], cmap = 'gray', vmin = 0.0, vmax = 1.0)
    hA.set(xticks = [], yticks = [])
    if figTitle is not None:
        hA.set_title(figTitle)

    return hF

In [ ]:
# Auxiliary Display Function
hDisplayImg = lambda imgIdx: DisplayImageArray(tR, imgIdx - 1, figTitle = f'Reconstruction by #{lSR[imgIdx - 1]:02d} Components')

In [ ]:
# Interactive Plot of the Unit Ball Interior

imgIdxSlider = IntSlider(value = 1, min = 1, max = numRec, step = 1, description = 'Image Index:', continuous_update = False, readout = True, readout_format = 'd', layout = Layout(width = '30%'))
interact(hDisplayImg, imgIdx = imgIdxSlider);

* <font color='red'>(**?**)</font> Explain the results when the number of components is zero.
<!-- The mean patch is tiled. -->
* <font color='red'>(**?**)</font> One suggests to use the system as a compression algorithm. Sending only the coefficients of `mUd.T @ mD`.  
  Calculate and the compression ratio as a function of the number of components.  
  How come it is not used as compression algorithm?
<!-- `pramK * paramK * numComp` + `paramK * paramK` for the mean patch * 4 Bytes (`Float32`). The decompression requires knowing `mUd` itself. -->